# TESSE NN Training Script

Use this to collect many rgb and segmented images within the simulator.

In [ ]:
from tesse.env import Env
from tesse.msgs import *

import time
import os
import pandas as pd
import numpy as np
import cv2
import tf.transformations

## Get spawn points from json file

The spawn-point file is located in `tesse_human_0x/tesse_human_0x_DATA/StreamingAssets/tesse_multiscene_[scene].points`

Replace `x` with the version; as of June 30 2020 the latest version is 09. Replace `[scene]` with the environment. For the office scene, this is "office2".

Second cell will tell you how many spawn points are available for the agent and each class of human. You may spawn more humans that there are spawn points; in this case humans will spawn on top of each other and disperse once commanded to move. Command humans to move by clicking on the simulator screen.

Note that you can capture additional spawn points for the agent by using spawn-capture-mode. More details [here](https://github.mit.edu/TESS/tesse-core#agent-spawn-points). Spawn yaw-angle is randomized so even with limited spawn points you can have novel images just by having different orientations.

In [ ]:
spawn_point_file = ""
spawn_points_df = pd.read_json(spawn_point_file)

In [ ]:
agent_points = []
female_points = []
male_points = []

for i in range(len(spawn_points_df)):
    name = spawn_points_df.iloc[i]['spawnPoints']['name']
    
    for point in spawn_points_df.iloc[i]['spawnPoints']['points']:
        if name == "tesse_multiscene_agent":
            agent_points.append(np.array([point[j] for j in ['x','y','z']]))
        elif name == "SMPL_Female_Autonomous":
            female_points.append(np.array([point[j] for j in ['x','y','z']]))
        elif name == "SMPL_Male_Autonomous":
            male_points.append(np.array([point[j] for j in ['x','y','z']]))

print("NOTE: " + str(len(agent_points)) + " points are available for image capture.")
print("NOTE: " + str(len(female_points)) + " points are available for female spawning.")
print("NOTE: " + str(len(male_points)) + " points are available for male spawning.")

## Setup simulator

Note: make sure simulator executable is running now. Set `scene_id` to 1 for the office scene.

Check to make sure the cameras are set up properly, and choose how many humans to spawn into the simulator.

In [ ]:
scene_id = 1

env = Env()
_ = env.request(SceneRequest(scene_id))

cameras=[
    (Camera.RGB_LEFT, Compression.OFF, Channels.THREE),
    (Camera.SEGMENTATION, Compression.OFF, Channels.THREE),
]

# These are the same settings used for uHumans2!
fov = 60
width = 720
height = 480
near_draw_dist = -0.05
far_draw_dist = 50
pos = [-0.05, 0, 0]
quat = [0, 0, 0, 1]

for camera in cameras:
    _ = env.request(SetCameraParametersRequest(camera[0], height, width, fov, near_draw_dist, far_draw_dist))

    response = env.request(CameraInformationRequest(camera[0]))
    if response is not None:
        print(response.metadata)

In [ ]:
num_females = 10
num_males = num_females

for i in range(num_females):
    _ = env.request(SpawnObjectRequest(0, ObjectSpawnMethod.RANDOM))

for i in range(num_males):
    _ = env.request(SpawnObjectRequest(1, ObjectSpawnMethod.RANDOM))

## Data collection

`save_img` and `show_img` are self-explanatory. Note that if `show_img` is enabled, you will have to click through images to advance to the next capture.

In [ ]:
num_imgs = 10
save_img = True
show_img = False
rgb_img_output_dir = ""
seg_img_output_dir = ""

for i in range(num_imgs):
    point = agent_points[np.random.randint(0, len(agent_points))]
    yaw = np.random.random() * 2.0 * np.pi
    quat = tf.transformations.quaternion_from_euler(0., yaw, 0.)
    
    env.send(Reposition(point[0], point[1], point[2], *quat))
    
    response = env.request(DataRequest(metadata=False, cameras=cameras))
    if response:
        # incoming encoding is bgr!
        img_rgb = cv2.cvtColor(response.images[0], cv2.COLOR_BGR2RGB)
        img_seg = cv2.cvtColor(response.images[1], cv2.COLOR_BGR2RGB)

        if save_img:
            filename_rgb = os.path.join(rgb_img_output_dir, "img_rgb_" + str(i) + ".jpg")
            filename_seg = os.path.join(seg_img_output_dir, "img_seg_" + str(i) + ".jpg")
            cv2.imwrite(filename_rgb, img_rgb)
            cv2.imwrite(filename_seg, img_seg)

        if show_img:
            for img in [img_rgb, img_seg]:
                cv2.imshow('img', img)
                cv2.waitKey(0)
                cv2.destroyAllWindows()